In [ ]:
// for local development
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' Datasets TranslationModels TextModels ModelSupport SummaryWriter LangMotionModels Checkpoints

In [ ]:
import TensorFlow
import TextModels
import TranslationModels
import Foundation
import FoundationXML
import ModelSupport
import Datasets
import SummaryWriter
import LangMotionModels
import Checkpoints
import PythonKit

In [ ]:
let np  = Python.import("numpy")

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

# Load dataset

In [ ]:
let maxTextSequenceLength =  20
let maxMotionLength =  100

In [ ]:
let datasetSize: DatasetSize = .mini
let batchSize = 150

In [ ]:
let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")
let motionDatasetURL = dataURL.appendingPathComponent("motion_dataset_v3.10Hz.\(datasetSize.rawValue)plist")
let langDatasetURL = dataURL.appendingPathComponent("labels_ds_v2.csv")

In [ ]:
let motionsURL = dataURL.appendingPathComponent("generated_motions/grouped_joints", isDirectory: true)
try! FileManager().createDirectory(at: motionsURL, withIntermediateDirectories: true)

In [ ]:
/// instantiate text processor
let vocabularyURL = dataURL.appendingPathComponent("vocab.txt")
let vocabulary: Vocabulary = try! Vocabulary(fromFile: vocabularyURL)
let tokenizer: Tokenizer = BERTTokenizer(vocabulary: vocabulary, caseSensitive: false, unknownToken: "[UNK]", maxTokenLength: nil)
let textProcessor = TextProcessor2(vocabulary: vocabulary, tokenizer: tokenizer, maxTextSequenceLength: maxTextSequenceLength, maxMotionLength: maxMotionLength)

In [ ]:
/// load dataset
print("\nLoading dataset...")

var dataset = try Lang2Motion(
    motionDatasetURL: motionDatasetURL,
    langDatasetURL: langDatasetURL,
    batchSize: batchSize
) { (example: Lang2Motion.Example) -> LangMotionBatch in    
    let singleBatch = textProcessor.preprocess(example: example)
    return singleBatch
}

print("Dataset acquired.")

In [ ]:
dataset.langRecsDict.keys.count

# group/reindex joints

In [ ]:
let ms = dataset.motionSamples[0]

In [ ]:
ms.annotations

# show motion image

In [ ]:
let decodedMotion = ms.motion

In [ ]:
let sentence = "sample_id=\(ms.sampleID), ann=\(ms.annotations[0])"

In [ ]:
let descaledMotion = dataset.scaler.inverse_transform(decodedMotion)

In [ ]:
print("  decodedMotion: min: \(decodedMotion.min()), max: \(decodedMotion.max())")

In [ ]:
print("  descaledMotion.shape: \(descaledMotion.shape)")
print("  descaledMotion: min: \(descaledMotion.min()), max: \(descaledMotion.max())")

In [ ]:
let jointNames = dataset.motionSamples[0].jointNames

In [ ]:
let prefix: String = "prefix"
var imageURL: URL? = motionsURL.appendingPathComponent("\(prefix).png")
let motion = MotionSample2.grouppedJoints(motion: descaledMotion, jointNames: ms.jointNames)
// let motion = descaledMotion
motionToImg(url: imageURL, motion: motion, motionFlag: nil, padTo: maxMotionLength, descr: "\(prefix), \(sentence)", cmapRange: 2.0)
print("Saved image: \(imageURL!.path)")

# save motion to mmm file

In [ ]:
let mmmXMLDoc = MMMWriter.getMMMXMLDoc(jointNames: jointNames, motion: descaledMotion)
let mmmURL = motionsURL.appendingPathComponent("\(prefix).mmm.xml")
try! mmmXMLDoc.xmlData(options: XMLNode.Options.nodePrettyPrint).write(to: mmmURL)
print("Saved motion: \(mmmURL.path)")